# 🔥 FIRE DETECTION - SIMPLE WORKING SOLUTION

**Run all cells in order to fix the fire detection problem**


In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# TensorFlow/Keras imports
try:
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    print("Using TensorFlow Keras")
except ImportError:
    from keras.preprocessing import image
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from keras.callbacks import EarlyStopping
    print("Using standalone Keras")

print("✅ All libraries imported successfully!")


Using TensorFlow Keras
✅ All libraries imported successfully!


In [2]:
# Load images from fire_dataset folder
print("🔥 LOADING DATA FROM FIRE_DATASET FOLDER")
print("=" * 50)

def load_images_simple(fire_dir, non_fire_dir, max_images=200):
    """Load images with simple preprocessing"""
    images = []
    labels = []
    
    # Load fire images
    fire_files = [f for f in os.listdir(fire_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:max_images//2]
    print(f"Loading {len(fire_files)} fire images...")
    
    for file in fire_files:
        try:
            img_path = os.path.join(fire_dir, file)
            img = image.load_img(img_path, target_size=(150, 150))
            img_array = image.img_to_array(img) / 255.0
            images.append(img_array)
            labels.append(1)  # Fire = 1
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    # Load non-fire images
    non_fire_files = [f for f in os.listdir(non_fire_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:max_images//2]
    print(f"Loading {len(non_fire_files)} non-fire images...")
    
    for file in non_fire_files:
        try:
            img_path = os.path.join(non_fire_dir, file)
            img = image.load_img(img_path, target_size=(150, 150))
            img_array = image.img_to_array(img) / 255.0
            images.append(img_array)
            labels.append(0)  # Non-fire = 0
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    return np.array(images), np.array(labels)

# Define paths
fire_path = 'fire_dataset/fire_images'
non_fire_path = 'fire_dataset/non_fire_images'

# Check if paths exist
if not os.path.exists(fire_path):
    print(f"❌ Fire path not found: {fire_path}")
    print("Available directories:")
    for item in os.listdir('.'):
        if os.path.isdir(item):
            print(f"  - {item}")
else:
    print(f"✅ Fire path found: {fire_path}")

if not os.path.exists(non_fire_path):
    print(f"❌ Non-fire path not found: {non_fire_path}")
else:
    print(f"✅ Non-fire path found: {non_fire_path}")

# Load images
X, y = load_images_simple(fire_path, non_fire_path, max_images=200)

print(f"\nLoaded {len(X)} images")
print(f"Shape: {X.shape}")
print(f"Fire images: {np.sum(y == 1)}")
print(f"Non-fire images: {np.sum(y == 0)}")


🔥 LOADING DATA FROM FIRE_DATASET FOLDER
✅ Fire path found: fire_dataset/fire_images
✅ Non-fire path found: fire_dataset/non_fire_images
Loading 100 fire images...
Loading 100 non-fire images...

Loaded 200 images
Shape: (200, 150, 150, 3)
Fire images: 100
Non-fire images: 100


In [3]:
# Split data into training and test sets
print("📊 SPLITTING DATA")
print("=" * 30)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape[0]} images")
print(f"Test set: {X_test.shape[0]} images")
print(f"Training - Fire: {np.sum(y_train == 1)}, Non-fire: {np.sum(y_train == 0)}")
print(f"Test - Fire: {np.sum(y_test == 1)}, Non-fire: {np.sum(y_test == 0)}")


📊 SPLITTING DATA
Training set: 160 images
Test set: 40 images
Training - Fire: 80, Non-fire: 80
Test - Fire: 20, Non-fire: 20


In [4]:
# Build simple CNN model
print("🏗️ BUILDING MODEL")
print("=" * 25)

def build_simple_model():
    """Build a simple CNN model"""
    model = Sequential([
        # Simple conv layers
        Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(2, 2),
        
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        
        # Flatten and dense
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build model
model = build_simple_model()
print("Model architecture:")
model.summary()

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"\nClass weights: {class_weight_dict}")
print(f"  Class 0 (NON-FIRE): {class_weight_dict[0]:.3f}")
print(f"  Class 1 (FIRE): {class_weight_dict[1]:.3f}")


🏗️ BUILDING MODEL
Model architecture:


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,183,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,207,457 (4.61 MB)

 Trainable params: 1,207,457 (4.61 MB)

 Non-trainable params: 0 (0.00 B)


Class weights: {0: np.float64(1.0), 1: np.float64(1.0)}
  Class 0 (NON-FIRE): 1.000
  Class 1 (FIRE): 1.000


In [5]:
# Train the model
print("🚀 TRAINING MODEL")
print("=" * 20)

callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)]

print("Starting training...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n✅ Model training completed!")

# Get final metrics
final_train_loss = history.history['loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_loss = history.history['val_loss'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"\nFinal Training Metrics:")
print(f"  Training Loss: {final_train_loss:.4f}")
print(f"  Training Accuracy: {final_train_acc:.4f}")
print(f"  Validation Loss: {final_val_loss:.4f}")
print(f"  Validation Accuracy: {final_val_acc:.4f}")


🚀 TRAINING MODEL
Starting training...
Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step - accuracy: 0.6250 - loss: 0.7406 - val_accuracy: 0.9000 - val_loss: 0.5791
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.9000 - loss: 0.3676 - val_accuracy: 0.9000 - val_loss: 0.2359
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 216ms/step - accuracy: 0.9187 - loss: 0.2262 - val_accuracy: 0.6250 - val_loss: 1.2727
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.8562 - loss: 0.3717 - val_accuracy: 0.8750 - val_loss: 0.2185
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.9625 - loss: 0.1417 - val_accuracy: 0.9250 - val_loss: 0.2134
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.9375 - loss: 0.1462 - val_accuracy: 0.9250 - val_loss: 0.2196
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.9375 - loss: 0.1321 - val_accuracy: 0.9500 - val_loss: 0.1302
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 163ms/step - accuracy: 0.9

In [6]:
# Test the model
print("🧪 TESTING MODEL")
print("=" * 20)

# Make predictions
y_pred_proba = model.predict(X_test, verbose=1)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(f"\nPrediction statistics:")
print(f"  Mean probability: {np.mean(y_pred_proba):.4f}")
print(f"  Min probability: {np.min(y_pred_proba):.4f}")
print(f"  Max probability: {np.max(y_pred_proba):.4f}")

print(f"\nPrediction distribution:")
unique, counts = np.unique(y_pred, return_counts=True)
for pred_class, count in zip(unique, counts):
    class_name = "FIRE" if pred_class == 1 else "NON-FIRE"
    percentage = (count / len(y_pred)) * 100
    print(f"  {class_name}: {count} predictions ({percentage:.1f}%)")

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel accuracy: {accuracy:.4f}")

# Check if predictions are balanced
fire_predictions = np.sum(y_pred == 1)
fire_percentage = (fire_predictions / len(y_pred)) * 100

print(f"\nPrediction balance:")
print(f"  Fire predictions: {fire_predictions}/{len(y_pred)} ({fire_percentage:.1f}%)")
print(f"  Non-fire predictions: {len(y_pred) - fire_predictions}/{len(y_pred)} ({100-fire_percentage:.1f}%)")

# Check if model is learning correctly
fire_prob_mean = np.mean(y_pred_proba[y_test == 1])
non_fire_prob_mean = np.mean(y_pred_proba[y_test == 0])

print(f"\nAverage prediction probabilities:")
print(f"  Fire images: {fire_prob_mean:.4f}")
print(f"  Non-fire images: {non_fire_prob_mean:.4f}")

if fire_prob_mean > non_fire_prob_mean:
    print("✅ Model is learning correctly - fire images have higher probabilities")
else:
    print("⚠️  Model is inverted - fire images have lower probabilities")


🧪 TESTING MODEL
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step

Prediction statistics:
  Mean probability: 0.5092
  Min probability: 0.0000
  Max probability: 1.0000

Prediction distribution:
  NON-FIRE: 21 predictions (52.5%)
  FIRE: 19 predictions (47.5%)

Model accuracy: 0.9750

Prediction balance:
  Fire predictions: 19/40 (47.5%)
  Non-fire predictions: 21/40 (52.5%)

Average prediction probabilities:
  Fire images: 0.9403
  Non-fire images: 0.0780
✅ Model is learning correctly - fire images have higher probabilities


In [7]:
# Test on actual images from dataset
print("🎯 TESTING ON ACTUAL IMAGES")
print("=" * 30)

def predict_fire(image_path, model, threshold=0.5):
    """Predict if image contains fire"""
    try:
        # Load image with simple preprocessing
        img = image.load_img(image_path, target_size=(150, 150))
        img_array = image.img_to_array(img) / 255.0
        
        # Make prediction
        prediction_proba = model.predict(np.expand_dims(img_array, axis=0), verbose=0)[0][0]
        prediction_class = 1 if prediction_proba > threshold else 0
        
        # Determine class name
        class_name = "FIRE" if prediction_class == 1 else "NON-FIRE"
        
        return {
            'image_path': image_path,
            'probability': float(prediction_proba),
            'prediction': prediction_class,
            'class_name': class_name,
            'confidence': float(abs(prediction_proba - 0.5) * 2)
        }
        
    except Exception as e:
        return {
            'image_path': image_path,
            'error': str(e),
            'probability': None,
            'prediction': None,
            'class_name': 'ERROR',
            'confidence': None
        }

# Get test images
test_fire_files = [f for f in os.listdir(fire_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:2]
test_non_fire_files = [f for f in os.listdir(non_fire_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:3]

print(f"Testing on {len(test_fire_files)} fire images and {len(test_non_fire_files)} non-fire images...")

correct_predictions = 0
total_predictions = 0

# Test fire images
for file in test_fire_files:
    image_path = os.path.join(fire_path, file)
    print(f"\n{'='*40}")
    print(f"Testing FIRE image: {file}")
    
    prediction_result = predict_fire(image_path, model)
    
    if 'error' not in prediction_result:
        print(f"Predicted: {prediction_result['class_name']}")
        print(f"Probability: {prediction_result['probability']:.4f}")
        print(f"Confidence: {prediction_result['confidence']:.2%}")
        
        if prediction_result['class_name'] == 'FIRE':
            print("✅ CORRECT - Fire image predicted as FIRE")
            correct_predictions += 1
        else:
            print("❌ WRONG - Fire image predicted as NON-FIRE")
        
        total_predictions += 1
    else:
        print(f"Error: {prediction_result['error']}")

# Test non-fire images
for file in test_non_fire_files:
    image_path = os.path.join(non_fire_path, file)
    print(f"\n{'='*40}")
    print(f"Testing NON-FIRE image: {file}")
    
    prediction_result = predict_fire(image_path, model)
    
    if 'error' not in prediction_result:
        print(f"Predicted: {prediction_result['class_name']}")
        print(f"Probability: {prediction_result['probability']:.4f}")
        print(f"Confidence: {prediction_result['confidence']:.2%}")
        
        if prediction_result['class_name'] == 'NON-FIRE':
            print("✅ CORRECT - Non-fire image predicted as NON-FIRE")
            correct_predictions += 1
        else:
            print("❌ WRONG - Non-fire image predicted as FIRE")
        
        total_predictions += 1
    else:
        print(f"Error: {prediction_result['error']}")

print(f"\n{'='*50}")
print(f"FINAL RESULTS: {correct_predictions}/{total_predictions} predictions were correct")

if correct_predictions == total_predictions:
    print("🎉 SUCCESS! Model is working perfectly!")
    print("   Fire images are correctly predicted as FIRE")
    print("   Non-fire images are correctly predicted as NON-FIRE")
elif correct_predictions > total_predictions // 2:
    print("✅ GOOD! Model is working well.")
    print("   Most predictions are correct")
else:
    print("⚠️  Model still has issues.")
    print("   This suggests a fundamental problem with the data or approach.")

print("=" * 50)


🎯 TESTING ON ACTUAL IMAGES
Testing on 2 fire images and 3 non-fire images...

Testing FIRE image: fire.753.png
Predicted: FIRE
Probability: 1.0000
Confidence: 100.00%
✅ CORRECT - Fire image predicted as FIRE

Testing FIRE image: fire.12.png
Predicted: FIRE
Probability: 0.9963
Confidence: 99.27%
✅ CORRECT - Fire image predicted as FIRE

Testing NON-FIRE image: non_fire.134.png
Predicted: NON-FIRE
Probability: 0.0000
Confidence: 100.00%
✅ CORRECT - Non-fire image predicted as NON-FIRE

Testing NON-FIRE image: non_fire.120.png
Predicted: NON-FIRE
Probability: 0.4297
Confidence: 14.07%
✅ CORRECT - Non-fire image predicted as NON-FIRE

Testing NON-FIRE image: non_fire.108.png
Predicted: NON-FIRE
Probability: 0.0411
Confidence: 91.78%
✅ CORRECT - Non-fire image predicted as NON-FIRE

FINAL RESULTS: 5/5 predictions were correct
🎉 SUCCESS! Model is working perfectly!
   Fire images are correctly predicted as FIRE
   Non-fire images are correctly predicted as NON-FIRE


In [8]:
# Save the model
print("💾 SAVING MODEL")
print("=" * 20)

model.save('fire_detection_model_fixed.h5')
print("✅ Model saved as 'fire_detection_model_fixed.h5'")

print("\n🔥 FIRE DETECTION FIX COMPLETED!")
print("You can now use the model to predict fire in new images.")
print("\nTo use the model in the future:")
print("1. Load the model: model = load_model('fire_detection_model_fixed.h5')")
print("2. Use the predict_fire function to classify new images")


💾 SAVING MODEL
✅ Model saved as 'fire_detection_model_fixed.h5'

🔥 FIRE DETECTION FIX COMPLETED!
You can now use the model to predict fire in new images.

To use the model in the future:
1. Load the model: model = load_model('fire_detection_model_fixed.h5')
2. Use the predict_fire function to classify new images
